## 01 Introduction
*  Import pandas, matplotlib, and numpy into the environment. Import the classes you need from scikit-learn as well.
*  Read __AmesHousing.txt__ into a pandas data frame.
*  For the following functions, we recommend creating them in the first few cells in the notebook. This way, you can add cells to the end of the notebook to do experiments and update the functions in these cells.
  *  Create a function named __transform_features()__ that, for now, just returns the __train__ data frame.
  *  Create a function named __select_features()__ that, for now, just returns the __Gr Liv Area__ and __SalePrice__ columns from the __train__ data frame.
  *  Create a function named __train_and_test()__ that, for now:
    *  Selects the first __1460__ rows from from __data__ and assign to __train__.
    *  Selects the remaining rows from __data__ and assign to __test__.
    *  Trains a model using all numerical columns except the __SalePrice__ column (the target column) from the data frame returned from __select_features()__
    *  Tests the model on the test set using and returns the RMSE value.

## Feature Engineering
*  As we mentioned earlier, we recommend adding some cells to explore and experiment with different features (before rewriting these functions).
*  The __transform_features()__ function shouldn't modify the __train__ data frame and instead return a new one entirely. This way, we can keep using __train__ in the experimentation cells.
*  Which columns contain less than 5% missing values?
  *  For numerical columns that meet this criteria, let's fill in the missing values using the most popular value for that column.
*  What new features can we create, that better capture the information in some of the features?
  *  An example of this would be the __years_until_remod__ feature we created in the last mission.
*  Which columns need to be dropped for other reasons?
  *  Which columns aren't useful for machine learning?
  *  Which columns leak data about the final sale?

## Feature Selection
*  Generate a correlation heatmap matrix of the numerical features in the training data set.
  *  Which features correlate strongly with our target column, __SalePrice__?
  *  Calculate the correlation coefficients for the columns that seem to correlate well with __SalePrice__. Because we have a pipeline in place, it's easy to try different features and see which features result in a better cross validation score.
*  Which columns in the data frame should be converted to the categorical data type? All of the columns marked as __nominal__ from the [documentation](DataDocumentation.txt) are candidates for being converted to categorical. Here are some other things you should think about:
  *  If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
  *  Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).
*  Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?
*  What are some ways we can explore which categorical columns "correlate" well with __SalePrice__?
  *  Read this post for some [potential strategies](https://machinelearningmastery.com/feature-selection-machine-learning-python/).
*  Update the logic for the __select_features()__ function. This function should take in the new, modified train and test data frames that were returned from __transform_features()__.

## 04 Train & Test
*  The optional __k__ parameter should accept integer values, with a default value of __0__.
*  When __k__ equals __0__, perform holdout validation (what we already implemented):
  *  Select the first __1460__ rows and assign to __train__.
  *  Select the remaining rows and assign to __test__.
  *  Train on train and test on __test__.
  *  Compute the RMSE and return.
*  When __k__ equals __1__, perform simple cross validation:
  *  Shuffle the ordering of the rows in the data frame.
  *  Select the first __1460__ rows and assign to __fold_one__.
  *  Select the remaining rows and assign to __fold_two__.
  *  Train on __fold_one__ and test on __fold_two__.
  *  Train on __fold_two__ and test on __fold_one__.
  *  Compute the average RMSE and return.
*  When __k__ is greater than __0__, implement k-fold cross validation using __k__ folds:
  *  Perform k-fold cross validation using __k__ folds.
  *  Calculate the average RMSE value and return this value.